# Data mining based on Google trends

Considerations
====================


### Data mining and Google trends
- This notebook demonstrates how to mining data based on a google trends exploration.
- This is useful for automating the data gathering process, considering that it is possible to save the information obtained on the web, rather than simply viewing it.

###  Libraries:
- selenium: open a browser (webdriver - http://chromedriver.chromium.org/downloads) and do the automation of actions such as clicks, scrolls, ect ...
- pandas: we will need to work with dataframes.
- seaborn: plotting and styling the graphs.
- matplotlib: plotting charts.
- time: sometimes we need to give a short wait between actions.
- os: library to access local files and folders.

In [4]:
# pip install selenium --user
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import time
import os

sns.set()
%matplotlib inline

In [11]:
# request the term to be searched
def inquiry():
    
    inquiry=str(input("Input the term to be searched... \n"))
    term_input = '%20'.join(inquiry.split())
    
    return term_input, inquiry 

In [13]:
# acquire the term
termo,pesquisa = inquiry()

Input the term to be searched... 
data sciencce
